# FITS to hepfile


In [ ]:
# first pip install astropy
!pip install astropy

In [ ]:
# imports
import os, glob
import hepfile as hf
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table

In [ ]:
# download some "small" public fits images, unzip them, and clean up the directory
# this downloads three fits files from noirlab with observations of the eagle nebula at 3 different wavelengths
!wget 'https://noirlab.edu/public/media/archives/education/zip/edu008.zip'
!unzip -o 'edu008.zip'
!unzip -o '*.zip'
!rm -rf '__MACOSX'
!rm *.zip
!rm *.psd

In [ ]:
# let's open the files
fitsfiles = glob.glob(os.path.join(os.getcwd(), '*.fits'))

fitsdata = []
for f in fitsfiles:
    hdu_list = fits.open(f)
    print(hdu_list.info()) # this shows all the info
    print()
    fitsdata.append(hdu_list)

In [ ]:
allinfo = []
imgs = []
for d in fitsdata:
    img = np.array(d[0].data)
    info = Table(d[1].data)

    print(info)
    plt.figure()
    plt.imshow(img.T, cmap='Greys_r')
    
    allinfo.append(info)
    imgs.append(img)

In [ ]:
# we can think of each of these images as "events" and create a dictionary from them

events = []
for img, meta in zip(imgs, allinfo):
    
    event_dict = {}
    for key in meta.keys():
        # these are all the singleton values
        event_dict[key] = meta[key].value[0]
        
    event_dict['data'] = {} # initialize a subdict for the image data
    for i, row in enumerate(img):
        event_dict['data'][f'row_{i}'] = row
    
    events.append(event_dict)

In [ ]:
# put this data in a hepfile
data = hf.dict_tools.dictlike_to_hepfile(events, 'eagle-nebula.h5', write_hepfile=True)

In [ ]:
key = "_SINGLETONSGROUPFORSTORAGE_"
datalist = ['COUNTER', 'CRVAL1', 'CRVAL2', 'CRPIX1', 'CRPIX2', 'CD1_1', 'CD1_2', 'CD2_1', 'CD2_2', 'DATAMIN', 'DATAMAX', 'MIR_REVR', 'ORIENTAT', 'FILLCNT', 'ERRCNT', 'FPKTTIME', 'LPKTTIME', 'CTYPE1', 'CTYPE2', 'DETECTOR', 'DEZERO', 'BIASEVEN', 'BIASODD', 'GOODMIN', 'GOODMAX', 'DATAMEAN', 'GPIXELS', 'SOFTERRS', 'CALIBDEF', 'STATICD', 'ATODSAT', 'DATALOST', 'BADPIXEL', 'OVERLAP', 'PHOTMODE', 'PHOTFLAM', 'PHOTZPT', 'PHOTPLAM', 'PHOTBW', 'MEDIAN', 'MEDSHADO', 'HISTWIDE', 'SKEWNESS', 'MEANC10', 'MEANC25', 'MEANC50', 'MEANC100', 'MEANC200', 'MEANC300', 'BACKGRND']
stra = np.string_(key)

mydataset = []
strb = np.string_("")
nvals = len(datalist)
for i, val in enumerate(datalist):
    strb += np.string_(val)
    if i < nvals - 1:
        strb += np.string_("__:__")
mydataset.append([stra, strb])
mydataset

In [ ]:
hepfile_data, bucket = hf.load('eagle-nebula.h5')

In [ ]:
hepfile_data['_GROUPS_']

In [ ]:
hf.awkward_tools.hepfile_to_awkward(hepfile_data)

In [ ]:
meta

In [ ]:
hepfile_data['_GROUPS_']

In [ ]:
events[0]